# Learning on Reber-Grammar

Warm up on the Reber-Grammar prediction problem using Keras

## Importing Libraries:

In [36]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, TimeDistributed, Masking
from keras.optimizers import RMSprop
from keras.utils import np_utils
import reber_utility as r_u
import numpy as np

some examples of the reber_utility:

In [3]:
# generate continuous reber-words
Data = r_u.generate_continuous_reberwords(10,10)

In [35]:
# convert Data to indices
indices = [[r_u.char2index[d] for d in Data[i]] for i in range(len(Data))]

In [34]:
# convert indices back to characters
chars = [[r_u.index2char[d] for d in indices[i]] for i in range(len(indices))]

Defining important parameters:

In [38]:
step_size = 50
n_step=100 #redefine this. IT is to make 1000 reber words

batch_size = 16
n_batch = 12 # total number
n_test_batch = 2 # testing part

train_idxes = np.array(range(0,(n_batch-n_test_batch)*batch_size))
test_idxes = np.array(range((n_batch-n_test_batch)*batch_size, n_batch*batch_size))

N_Epoch = 20

generate and vectorize Data:

In [43]:
D = r_u.generate_continuous_reberwords(n_batch*batch_size,step_size*n_step+1 - 20)

#vectorize (should be also put into utility ;)

X = -1*np.ones((n_batch*batch_size, step_size*n_step+1, len(r_u.char2index)), dtype=np.float64)
for j, word in enumerate(D):
    categorized = np_utils.to_categorical([r_u.char2index[c] for c in word])
    X[j,:min(len(word,),step_size*n_step+1)] =  categorized[:min(len(word),step_size*n_step+1)]

Y = np.roll(X,-1,axis=1)
X = np.delete(X,-1,axis=1)
Y = np.delete(Y,-1,axis=1)

## Building the model
For the masking layer the value "-1" was chosen since to categorical already uses the values "0" and "1"

In [41]:
model = Sequential()
model.add(Masking(mask_value= -1., batch_input_shape=(batch_size,step_size,len(chars))))
model.add(LSTM(128, return_sequences=True, batch_input_shape=(batch_size,step_size,len(chars)),stateful=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

## Train model
train model and generate metrics

In [ ]:
metrics_train = np.zeros((N_Epoch,len(model.metrics_names)))
metrics_test = np.zeros((N_Epoch,len(model.metrics_names)))

print('Starting to learn:')
for i in range(N_Epoch):
    print('------- {} out of {} Epoch -----'.format(i+1,N_Epoch))

    ## Epochs should take all data; batches presented random, reset at each end of batch_size
    np.random.shuffle(train_idxes)
    batches_idxes = np.reshape(train_idxes, (-1,batch_size))
    for j, batch  in enumerate(batches_idxes):
        print('batch {} of {}'.format(j+1,n_batch-n_test_batch))
        for k in range(n_step):
            metrics_train[i] += model.train_on_batch(X[batch,k*step_size:(k+1)*(step_size)], Y[batch,k*step_size:(k+1)*step_size]) #python 0:3 gives 0,1,2 (which is not intuitive at all)
        model.reset_states() 

    test_batch_idxes = np.reshape(test_idxes,(-1,batch_size))
    for test_batch in test_batch_idxes:
        for k in range(n_step):
            metrics_test[i] += model.test_on_batch(X[test_batch,k*step_size:(k+1)*(step_size)], Y[test_batch,k*step_size:(k+1)*step_size])
        model.reset_states() 

    print('Train results:\t {} \n \t {}'.format(model.metrics_names, metrics_train[i]/(n_batch-n_test_batch) ))
    print('Test results:\t {} \n \t {}'.format(model.metrics_names, metrics_test[i]/(n_test_batch) ))

don't forget to save everything ;)

In [ ]:
model.save('embedCerg_model.h5')
np.save('embedXdata.npy',X)
np.save('embedydata.npy',Y)
np.save('embedTrainMetrics', metrics_train)
np.save('embedTestMetrics', metrics_test)